In [ ]:
import pulp
import matplotlib.pyplot as plt 
import numpy as npa 

In [ ]:
# Check IR and IC constraints
eps = 1e-4
for i in range(N):
    util_i = agents[i][0] * allocations[i] - agents[i][1] * prices[i] - times[i]
    if util_i + eps < 0: print(f"IR violated at {i}")
    for j in range(N):
        if i != j:
            util_j = agents[i][0] * allocations[j] - agents[i][1] * prices[j] - times[j]
            if util_i + eps < util_j:
                print(f"IC violated at ({i}, {j})")
                print(f"Utility of i: {util_i}; Utility of j: {util_j}")


### Indifference Curves

Below we will complete an exercise to plot the indifference regions induced by the menus.

In [ ]:
# convert menus to array 
menus_arr = np.vstack(list(menus.keys()))

# define function to get optimal menu given a row
def get_optimal_menu(row):
    row = row.copy()
    row[1] *= -1 
    row = np.concatenate((row, [-1]))
    val = menus_arr.dot(row)
    return np.argmax(val)

# define grid
x = np.linspace(0, 101, 1000)
y = np.linspace(0, 101, 1000)
xv, yv = np.meshgrid(x, y)
xygrid = np.vstack((xv.flatten(), yv.flatten())).T

In [ ]:
# get optimal menu at each point in grid, note this isn't most efficient way to do this
opt_men = np.apply_along_axis(get_optimal_menu, 1, xygrid)

In [ ]:
# plot indifference curves

fig, ax = plt.subplots(1, 1, figsize=(4,3))
for i, b in enumerate(menus): 
    print(f"Menu {i}: {np.round(b[0], digits)} got; {np.round(b[1], digits)} spent; {np.round(b[2], digits)} burned")
    pts_i = np.where(opt_men == i)
    ax.scatter(xygrid[pts_i, 0], xygrid[pts_i, 1], color = colors[i], label = f"Menu {i}", s = 1)

ax.set_xlabel("Value of good")
ax.set_ylabel("Value of money")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.show()

### Function (LP)

In [ ]:
def run_lp_fixed_t(
    agents, 
    M,
    alpha, 
    T,
    beta = 1
):
    # initialize LP 
    prob = pulp.LpProblem("Welfare_Maximization", pulp.LpMaximize)
    # Create variables
    x = pulp.LpVariable.dicts("Allocation", range(N), 0, 1)  # allocation per agent
    p = pulp.LpVariable.dicts("Price", range(N), 0)  # price per agent
    choose_T = pulp.LpVariable.dicts("Choose_T", range(N), cat='Binary')  # binary variable to decide t
    # Objective function
    prob += pulp.lpSum([beta * (agents[i][0] * x[i] - agents[i][1] * p[i] - choose_T[i] * T) + alpha * p[i] for i in range(N)])
    # Constraint: sum of the allocations <= M
    prob += pulp.lpSum(x) <= M
    # Constraints: individual rationality and incentive compatibility
    for i in range(N):
        prob += agents[i][0] * x[i] - agents[i][1] * p[i] - choose_T[i] * T >= 0  # utility > 0
        for j in range(N):
            if i != j:
                prob += agents[i][0] * x[i] - agents[i][1] * p[i] - choose_T[i] * T >= agents[i][0] * x[j] - agents[i][1] * p[j] - choose_T[j] * T  # utility from own allocation >= utility from other's allocation
    # Solve problem
    prob.solve(pulp.PULP_CBC_CMD(msg=0))

    # Store optimal values 
    allocations = []
    prices = []
    times = []
    nonzero_i = []
    menus = {}
    menus[0.0, 0.0, 0.0] = []
    for i in range(N):
        t = T if choose_T[i].varValue == 1 else 0
        bundle = (np.round(x[i].varValue, 3), np.round(p[i].varValue, 3), t)
        allocations.append(bundle[0])
        prices.append(bundle[1])
        times.append(bundle[2])
        if x[i].varValue > 0: nonzero_i.append(i)
    
        if bundle not in menus: 
            menus[bundle] = []
    
        menus[bundle].append(i)

    return menus, (allocations, prices, times)

In [ ]:
def run_ilp_fixed_t(
    agents, 
    M,
    alpha, 
    T,
    beta = 1
):
    # initialize LP 
    prob = pulp.LpProblem("Welfare_Maximization", pulp.LpMaximize)
    # Create variables
    x = pulp.LpVariable.dicts("Allocation", range(N), 0, 1, cat = "Integer")  # allocation per agent
    p = pulp.LpVariable.dicts("Price", range(N), 0)  # price per agent
    choose_T = pulp.LpVariable.dicts("Choose_T", range(N), cat='Binary')  # binary variable to decide t
    # Objective function
    prob += pulp.lpSum([beta * (agents[i][0] * x[i] - agents[i][1] * p[i] - choose_T[i] * T) + alpha * p[i] for i in range(N)])
    # Constraint: sum of the allocations <= M
    prob += pulp.lpSum(x) <= M
    # Constraints: individual rationality and incentive compatibility
    for i in range(N):
        prob += agents[i][0] * x[i] - agents[i][1] * p[i] - choose_T[i] * T >= 0  # utility > 0
        for j in range(N):
            if i != j:
                prob += agents[i][0] * x[i] - agents[i][1] * p[i] - choose_T[i] * T >= agents[i][0] * x[j] - agents[i][1] * p[j] - choose_T[j] * T  # utility from own allocation >= utility from other's allocation
    # Solve problem
    prob.solve(pulp.PULP_CBC_CMD(msg=0))

    # Store optimal values 
    allocations = []
    prices = []
    times = []
    nonzero_i = []
    menus = {}
    menus[0.0, 0.0, 0.0] = []
    for i in range(N):
        t = T if choose_T[i].varValue == 1 else 0
        bundle = (np.round(x[i].varValue, 3), np.round(p[i].varValue, 3), t)
        allocations.append(bundle[0])
        prices.append(bundle[1])
        times.append(bundle[2])
        if x[i].varValue > 0: nonzero_i.append(i)
    
        if bundle not in menus: 
            menus[bundle] = []
    
        menus[bundle].append(i)

    return menus, (allocations, prices, times)

In [ ]:
def run_lp(
    agents, 
    M,
    alpha, 
    beta = 1
):
    # initialize LP 
    prob = pulp.LpProblem("Welfare_Maximization", pulp.LpMaximize)
    # Create variables
    x = pulp.LpVariable.dicts("Allocation", range(N), 0, 1)  # allocation per agent
    p = pulp.LpVariable.dicts("Price", range(N), 0)  # price per agent
    t = pulp.LpVariable.dicts("Time", range(N), 0)  # time per agent
    # Objective function
    prob += pulp.lpSum([beta * (agents[i][0] * x[i] - agents[i][1] * p[i] - t[i]) + alpha * p[i] for i in range(N)])
    # Constraint: sum of the allocations <= M
    prob += pulp.lpSum(x) <= M
    for i in range(N):
        prob += agents[i][0] * x[i] - agents[i][1] * p[i] - t[i] >= 0  # utility > 0
    # Constraints: individual rationality and incentive compatibility
    for i in range(N):
        prob += agents[i][0] * x[i] - agents[i][1] * p[i] - t[i] >= 0  # utility > 0
        for j in range(N):
            if i != j:
                prob += agents[i][0] * x[i] - agents[i][1] * p[i] - t[i] >= agents[i][0] * x[j] - agents[i][1] * p[j] - t[j]  # utility from own allocation >= utility from other's allocation
    # Solve problem
    prob.solve(pulp.PULP_CBC_CMD(msg=0))

    # Store optimal values 
    allocations = []
    prices = []
    times = []
    nonzero_i = []
    menus = {}
    menus[0.0, 0.0, 0.0] = []
    for i in range(N):
        bundle = (np.round(x[i].varValue, 3), np.round(p[i].varValue, 3), np.round(t[i].varValue, 3))
        allocations.append(bundle[0])
        prices.append(bundle[1])
        times.append(bundle[2])
        if x[i].varValue > 0: nonzero_i.append(i)
    
        if bundle not in menus: 
            menus[bundle] = []
    
        menus[bundle].append(i)

    return menus, (allocations, prices, times)

In [ ]:
def run_ilp(
    agents, 
    M,
    alpha, 
    beta = 1
):
    # initialize LP 
    prob = pulp.LpProblem("Welfare_Maximization", pulp.LpMaximize)
    # Create variables
    x = pulp.LpVariable.dicts("Allocation", range(N), 0, 1, cat = "Integer")  # allocation per agent
    p = pulp.LpVariable.dicts("Price", range(N), 0)  # price per agent
    t = pulp.LpVariable.dicts("Time", range(N), 0)  # time per agent
    # Objective function
    prob += pulp.lpSum([beta * (agents[i][0] * x[i] - agents[i][1] * p[i] - t[i]) + alpha * p[i] for i in range(N)])
    # Constraint: sum of the allocations <= M
    prob += pulp.lpSum(x) <= M
    for i in range(N):
        prob += agents[i][0] * x[i] - agents[i][1] * p[i] - t[i] >= 0  # utility > 0
    # Constraints: individual rationality and incentive compatibility
    for i in range(N):
        prob += agents[i][0] * x[i] - agents[i][1] * p[i] - t[i] >= 0  # utility > 0
        for j in range(N):
            if i != j:
                prob += agents[i][0] * x[i] - agents[i][1] * p[i] - t[i] >= agents[i][0] * x[j] - agents[i][1] * p[j] - t[j]  # utility from own allocation >= utility from other's allocation
    # Solve problem
    prob.solve(pulp.PULP_CBC_CMD(msg=0))

    # Store optimal values 
    allocations = []
    prices = []
    times = []
    nonzero_i = []
    menus = {}
    menus[0.0, 0.0, 0.0] = []
    for i in range(N):
        bundle = (np.round(x[i].varValue, 3), np.round(p[i].varValue, 3), np.round(t[i].varValue, 3))
        allocations.append(bundle[0])
        prices.append(bundle[1])
        times.append(bundle[2])
        if x[i].varValue > 0: nonzero_i.append(i)
    
        if bundle not in menus: 
            menus[bundle] = []
    
        menus[bundle].append(i)

    return menus, (allocations, prices, times)

In [ ]:
def plot_lp(agents, menus):
    N = len(menus)
    # let's visualize the menus by plots 
    cm = plt.get_cmap('gist_rainbow')
    colors = [cm(1.*i/N) for i in range(N)]
    
    fig, ax = plt.subplots(1, 1, figsize=(4,3))
    digits = 4
    for i, b in enumerate(menus): 
        print(f"Menu {i}: {np.round(b[0], digits)} got; {np.round(b[1], digits)} spent; {np.round(b[2], digits)} burned")
        agents_i = menus[b]
        ax.scatter(agents[agents_i, 0], agents[agents_i, 1], color = colors[i], alpha = 0.25, label = f"Menu {i}")
    
    #ax.set_xlim(0, 101)
    ax.set_xlabel("Value of good")
    ax.set_ylabel("Value of money")
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.tight_layout()
    plt.show()

### Poor v. Rich agents

In [ ]:
N = 100
agents = np.zeros((N, 2))
# all agents have same value of good 
agents[: N // 2, 0] = np.random.randint(60, 101, N // 2)
agents[N // 2 :, 0] = np.random.randint(2, 40, N // 2)
# half agents are poor
agents[: N // 2, 1] = np.random.randint(60, 101, N // 2)
# other half are rich 
agents[N // 2 :, 1] = np.random.randint(2, 40, N // 2)
#agents[N // 2 :, 1] = 10

### Define agents by r

In [ ]:
r = np.concatenate((np.random.uniform(0.1, 1, N // 4), np.random.uniform(1, 5, 3 * N // 4)))
agents[:, 1] = np.random.uniform(1, 11, N)
agents[:, 0] = r * agents[:, 1]

In [ ]:
menus, _ = run_lp(agents, M = 30, alpha = np.mean(agents[:, 1]), beta = 1)
plot_lp(agents, menus)

In [ ]:
menus, _ = run_ilp(agents, M = 30, alpha = np.mean(agents[:, 1]), beta = 1)
plot_lp(agents, menus)

### No correlation

In [ ]:
agents[:, 1] = np.random.uniform(1, 101, N)
agents[:, 0] = np.random.uniform(1, 101, N)

In [ ]:
menus, _ = run_lp(agents, M = 30, alpha = np.mean(agents[:, 1]), beta = 1)
plot_lp(agents, menus)

In [ ]:
menus, _ = run_ilp(agents, M = 30, alpha = np.mean(agents[:, 1]), beta = 1)
plot_lp(agents, menus)